# 0 YouTube Crawling

#### 유튜브 내 검색 결과의 콘텐츠 정보 자동 Scrap을 통하여 '제페토커스텀' 과 관련한 영상과 조회수를 알아본다
https://github.com/park-gb/youtube-content-scaper

# 1 라이브러리 로드 

In [1]:
# 크롤링에 필요한 라이브러리를 불러온다 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\LG\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


In [3]:
#판다스를 통하여 수집한 데이터들을 데이터프레임으로 만들어 줄 예정이다
import time
import random
import pandas as pd

# 2 무한 스크롤 함수 셋팅

## 1) 기능 - 콘텐츠 로딩을 충분히 기다리며 스크롤이 불가할 때 까지 스크롤을 무한 반복하는 함수

## 2) 역할 - 유튜브 웹 페이지는 스크롤을 해야 새로운 콘텐츠 정보를 제공하기 때문에, 모든 검색 결과를 확인하기 위해서는 무한 스크롤 기능이 필요하다

In [4]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

# 3 데이터 스크랩

In [5]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = '제페토커스텀'.replace(' ', '+')

In [6]:
driver = webdriver.Chrome(service=service)
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)
# 무한 스크롤 함수 실행
scroll()

스크롤 완료


In [19]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 4 데이터 추출

In [20]:
# 모든 콘텐츠 정보
content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
# 콘텐츠 제목만 추출
content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
# 콘텐츠 url만 추출
content_total_view = list(map(lambda data: "https://youtube.com" + data["href"], content_total))
# 딕셔너리 포맷팅
content_total_dict = {'title' : content_total_title, 'view': content_total_view}

# 5 데이터프레임 저장

In [21]:
df = pd.DataFrame(content_total_dict)
df.to_csv("data/content_total.csv", encoding='utf-8-sig')

In [22]:
df

,title,view
0,[제페토 커스텀] 청초한 제페토 커스텀 하는법! (설명 참고),https://youtube.com/watch?v=Dj0lFDGnp0c
1,[제페토 커스텀] 도도한 고양이상 제페토 커스텀 하는법 (설명 참고),https://youtube.com/watch?v=oktexgUqLaQ
2,[ 제페토 커스텀 2탄 ] 동글동글 귀여운 제페토 커스텀 / 제페토 커스텀 / 제페...,https://youtube.com/watch?v=qdQYM-vmNzU
3,[제페토] 성숙한 커스텀 얼굴 만들기!,https://youtube.com/watch?v=5tnYs9EdILU
4,여우상 커스텀 (팔로우하고 따라하세요 o._911),https://youtube.com/watch?v=1SuFqDb4dA8
...,...,...
184,제페토 커스텀,https://youtube.com/shorts/ct3d8GbYK3w
185,여캐 제페토 커스텀강좌💕/인싸커스텀/예쁜커스텀/제페토커스텀/월령커스텀,https://youtube.com/watch?v=plEdMCkYLqg
186,제페토 커스텀-귀여운 얼굴 만들기,https://youtube.com/watch?v=bkC7VZtMock
187,[제페토 커스텀] 귀여운 아가 커스텀 🤍/zepeto / 제페토 커스텀 #제페토 #...,https://youtube.com/watch?v=veIn5T-pphM
